### Dependencies

In [241]:
import pandas as pd
import numpy as np
import json
import re
from nltk.corpus import stopwords
import nltk
from sklearn.externals import joblib

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from splinter import Browser
import os
from bs4 import BeautifulSoup as bs
import requests
import time

### Scraping From Webpage

In [242]:
filepath = os.path.join("CyberPsyche_WebPage", "templates", "index.html")
with open(filepath) as file:
    html = file.read()

In [243]:
# Create a Beautiful Soup object
soup = bs(html, 'html.parser')

In [244]:
# Print formatted version of the soup
print(soup.prettify())

<!DOCTYPE doctype html>
<html lang="en">
 <head>
  <!-- Required meta tags -->
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <!-- Style.css Stylesheet -->
  <link href="{{url_for('static', filename='css/style.css')}}" rel="stylesheet"/>
  <!-- Bootstrap CSS -->
  <link crossorigin="anonymous" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" integrity="sha384-ggOyR0iXCbMQv3Xipma34MD+dH/1fQ784/j6cY/iJTQUOhcWr7x9JvoRxT2MZw1T" rel="stylesheet"/>
  <!-- Google WebFonts -->
  <link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet"/>
  <!-- d3 -->
  <script src="https://d3js.org/d3.v5.min.js">
  </script>
  <title>
   CyberPsyche
  </title>
 </head>
 <body>
  <!-- Navigation -->
  <nav class="navbar navbar-expand-lg navbar-light bg-light">
   <!-- Brand -->
   <a class="navbar-brand mr-4" href="/">
    <div class="logo">
     CyberPsyche
    </div>
   </a>
   <!

In [245]:
# Print only the comments
comments = []
for item in soup.body.find_all("div", class_="comment"):
    comments.append(item.p.text)
    
print(comments)

["What a wonderful day! It's BEAUTIFUL outside isn't it???", 'Fuck you and the horse you rode in on!', 'OMG LOOOOOVVVVe!', "You are the BEST son'bitch I know", 'I AM GOING TO KILL YOU!!!', 'Hahaha! You are seriously the worst!']


### Pre Processing

In [246]:
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

In [247]:
df = pd.read_json("CyberPsyche_WebPage/Resources/Data_cyb.json", lines = True, orient = "columns")

In [248]:
rating = []

for i in df["annotation"]:
    rating.append(int(i["label"][0]))
    
df["rating"] = rating

In [249]:
tweets = pd.read_csv("CyberPsyche_WebPage/Resources/Test_Twitter_Comments.csv")
tweets.tail()

,content,rating
96,That is someone who does it from their heart. ...,1
97,Absolutely applaud your work to secure freedom...,0
98,You'll never learn it till you actually live i...,1
99,Nothing on the reinstatement of federal Capito...,1
100,Crickets,0


In [250]:
new_df1 = df[["content", "rating"]]

In [251]:
new_df = pd.concat([new_df1,tweets])

In [252]:
X_all = [i[1]["content"] for i in new_df.iterrows()]
y = [i[1]["rating"] for i in new_df.iterrows()]

reviews_train_clean = preprocess_reviews(X_all)

In [253]:
tweets_list = [i for i in comments]
# rating_list = [i[1]["rating"] for i in tweets.iterrows()]

In [254]:
twitter_cleaned = preprocess_reviews(tweets_list)

# Fitting Model

In [255]:
stop_words = stopwords.words('english')
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 3), stop_words=stop_words)
ngram_vectorizer.fit(reviews_train_clean)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), preprocessor=None,
        stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs',... 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [256]:
X = ngram_vectorizer.transform(reviews_train_clean)

In [257]:
X.shape

(20102, 163735)

In [258]:
final = LinearSVC(tol=.000001,C=0.06)
final.fit(X, y)
# print ("Final Accuracy: %s" 
#        % accuracy_score(y_test, final.predict(X_test)))

LinearSVC(C=0.06, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=1e-06,
     verbose=0)

In [259]:
print(twitter_cleaned)

["what a wonderful day it's beautiful outside isn't it", 'fuck you and the horse you rode in on', 'omg looooovvvve', "you are the best son'bitch i know", 'i am going to kill you', 'hahaha you are seriously the worst']


In [260]:
tws = ngram_vectorizer.transform(twitter_cleaned)
tws.shape

(6, 163735)

In [264]:
predictions = final.predict(tws)
print(predictions)
type(predictions)

[0 1 0 1 0 0]


numpy.ndarray

In [263]:
for i in twitter_cleaned:
    print(i)

what a wonderful day it's beautiful outside isn't it
fuck you and the horse you rode in on
omg looooovvvve
you are the best son'bitch i know
i am going to kill you
hahaha you are seriously the worst
